# Simple CNN in Pytorch

This is a simple implementation and training of a Convolutional Neural Network in Pytorch. 

![Neuron Diagram](./../resources/images/cnn1.png)

In [78]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

## Architecture

The chosen architecture is LeNet-5

![LeNet](./../resources/images/lenet1.png)

![LeNet](./../resources/images/lenet2.png)

In [79]:
class CNN_LeNet5(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        """
        Define the layers of the convolutional neural network.

        Parameters:
            in_channels: int
                The number of channels in the input image. For MNIST, this is 1 (grayscale images).
            num_classes: int
                The number of classes we want to predict, in our case 10 (digits 0 to 9).
        """
        super(CNN_LeNet5, self).__init__()

        # First convolutional layer: 1 input channel, 6 output channels, 5x5 kernel, stride 1, padding 0
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5, stride=1, padding=0)
        # Avg pooling layer: 2x2 window, stride 2
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        # Second convolutional layer: 6 input channels, 16 output channels, 5x5 kernel, stride 1, padding 0
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        # First fully connected layer: 16*4*4 input features (after two 2x2 poolings), 120 output features
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        # Second fully connected layer: 120 input features, 84 output features
        self.fc2 = nn.Linear(120, 84)
        # Third fully connected layer: 84 input features, 10 output features (num_classes)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        """
        Define the forward pass of the neural network.

        Parameters:
            x: torch.Tensor
                The input tensor.

        Returns:
            torch.Tensor
                The output tensor after passing through the network.
        """
        x = F.tanh(self.conv1(x))  # Apply first convolution and tanh activation
        x = self.pool(x)           # Apply avg pooling
        x = F.tanh(self.conv2(x))  # Apply second convolution and tanh activation
        x = self.pool(x)           # Apply avg pooling
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.tanh(self.fc1(x))    # Apply first fully connected layer
        x = F.tanh(self.fc2(x))    # Apply second fully connected layer
        x = F.softmax(self.fc3(x), dim=-1) # Apply third fully connected layer
        return x

## Layers explanation

### Convolutions

There are different compnoents related to convolutions:

Filters (Kernels):

- Filters are small matrices that slide over the input image and perform element-wise multiplications followed by summation. Each filter is designed to detect a specific feature in the input image.

- For example, a filter might detect horizontal edges, vertical edges, or more complex textures.

Stride:

- Stride is the step size with which the filter moves across the input image.

- A stride of 1 means the filter moves one pixel at a time, both horizontally and vertically.

- A larger stride reduces the size of the feature map because the filter skips more pixels. For instance, a stride of 2 means the filter moves two pixels at a time, effectively down-sampling the feature map.

Padding:

- Padding involves adding extra pixels around the input image’s border. These extra pixels are typically set to zero (zero-padding).

- Padding ensures that the filter fits properly over the image, especially at the edges. Without padding, the feature map’s size reduces after each convolution operation.

- For example, if you have a 5x5 input image and a 3x3 filter with no padding, the resulting feature map will be 3x3. With padding of 1, the feature map remains the same size as the input.

Feature Map:

- A feature map is the output of a convolutional layer after applying filters to the input image.

- Each feature map corresponds to a different filter and captures different features from the input.

- Stacking multiple feature maps together forms a multi-channel output that serves as the input for the next layer.


### Pooling

Pooling layers reduce the spatial dimensions of the feature maps, which helps in making the network computationally efficient and reducing overfitting. There are two main types of pooling:

Max Pooling:
- Max pooling takes the maximum value from each patch of the feature map.

- For example, in a 2x2 max pooling operation, the maximum value from each 2x2 block of the feature map is taken to create a new, smaller feature map.

- This operation reduces the size of the feature map by half, both horizontally and vertically, but retains the most prominent features.

Average Pooling:
- Average pooling takes the average value from each patch of the feature map.

- Similar to max pooling, but instead of the maximum value, it takes the average value from each block.

- This can be useful in different contexts, though max pooling is more common in practice.

## Set Up Device

In [88]:
def choose_device():
    """
    Choose the device to run the model on.

    Returns:
        torch.device
            The device to run the model on.
    """
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')
    
device = choose_device()

## Define Hyperparameters

In [81]:
input_size = 784  # 28x28 pixels (not directly used in CNN)
num_classes = 10  # digits 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 10  # Reduced for demonstration purposes

## Load data

Since this is a simple demonstration, the MNIST data from torchvision.datasets will suffice.

In [82]:
train_dataset = datasets.MNIST(root="dataset/", download=True, train=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", download=True, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [83]:
model = CNN_LeNet5(in_channels=1, num_classes=num_classes).to(device)

In [84]:
# initialize the model
#model = CNN(in_channels=1, num_classes=num_classes).to(device)

## Loss and Optimizer

In [85]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training the Network

In [86]:
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        # Move data and targets to the device (GPU/CPU)
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass: compute the model output
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass: compute the gradients
        optimizer.zero_grad()
        loss.backward()

        # Optimization step: update the model parameters
        optimizer.step()

Epoch [1/10]


100%|██████████| 938/938 [00:04<00:00, 223.26it/s]


Epoch [2/10]


100%|██████████| 938/938 [00:04<00:00, 210.70it/s]


Epoch [3/10]


...

100%|██████████| 938/938 [00:04<00:00, 196.84it/s]


Epoch [9/10]


100%|██████████| 938/938 [00:04<00:00, 206.41it/s]


Epoch [10/10]


100%|██████████| 938/938 [00:04<00:00, 226.17it/s]


## Check accuracy 



In [87]:
def check_accuracy(loader, model):
    """
    Checks the accuracy of the model on the given dataset loader.

    Parameters:
        loader: DataLoader
            The DataLoader for the dataset to check accuracy on.
        model: nn.Module
            The neural network model.
    """
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")
    
    model.train()  # Set the model back to training mode

# Final accuracy check on training and test sets
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59476/60000 with accuracy 99.13%
Checking accuracy on test data
Got 9856/10000 with accuracy 98.56%
